### <font color=red> **This notebook has three objectives:**
#### **1. Upload the pretrained model from Google Drive to Google Cloud Storage** (_if the model trained on the google cloud platform, this step could be igored_).
#### **2. Prepare the earth engine model through EEification.**
#### **3. Host the EEified model to the AI Platform.**

In [ ]:
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')


### <font color=red> **！Note: the runtime-version setting (supported tensorflow version in AI platform) should be consistency with the Tensorflow version in the colab, and we use Tensorflow 2.2.0 here!**

In [ ]:
# !pip install tensorflow==2.2.0
import tensorflow as tf
print(tf.__version__)

In [ ]:
import os
os.chdir("/content/drive/My Drive/Earth-Engine-with-Deep-Learning")
import matplotlib.pyplot as plt
import folium
import json


In [ ]:
## Parameters configuration
# Cloud project ID and the bucket
Project = 'my-project-20200813'
Bucket = 'earth-engine-bucket-1'
# Model path
Model_drive_path = 'models/pretrain/unet_v1/model'
Model_gs_dir = 'gs://' + Bucket + '/google_colab_train'
Model_gs_path = Model_gs_dir + '/impervious_unet'
# Put the EEified model next to the trained model directory.
EEified_path = Model_gs_dir + '/eeified_impervious_unet'

# model name and version on the AI platform.
Model_gs_name = 'impervious_unet'
Version = 'v1'


### **1. Upload the pretrained model from Google Drive to Google Cloud Storage.**

In [ ]:
## Authenticate to Google Earth engine
from google.colab import auth
auth.authenticate_user()

In [ ]:
## load model, the trained model should in the tensorflow 2.2.0 version
model = tf.keras.models.load_model(Model_drive_path)  # 
# model.summary()
## upload the pre-trained model to google cloud platform
model.save(Model_gs_path, save_format='tf')


### **2. Pretrained model EEification**

EEIfication prepares the model for hosting on Google AI Platform. First, get (and SET) input and output names of the nodes. Keep the input name of 'array', which is how you'll pass data into the model (as an array image). and change the ouput name to something that makes sense for the specific model.  

In [ ]:
## Authenticate to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(Model_gs_path, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "impervious"}) + "'"
print(input_dict)
print(output_dict)

'{"serving_default_input_3:0": "array"}'
'{"StatefulPartitionedCall:0": "impervious"}'


### **Run the EEifier**

The actual EEification is handled by the `earthengine modle prepare` command.  Note that you will need to set your Cloud Project prior to running the command.

In [ ]:
# You need to set the project before using the model prepare command.
!earthengine set_project {Project}
!earthengine model prepare --source_dir {Model_gs_path} --dest_dir {EEified_path} --input {input_dict} --output {output_dict}


### **3. Host the EEified model on AI Platform**

Now there is another TensorFlow `SavedModel` stored in `EEIFIED_DIR` ready for hosting by AI Platform.  Do that from the `gcloud` command line tool, installed in the Colab runtime by default.  Note that the `MODEL_NAME` must be unique.  If you already have a model by that name, either name a new model or a new version of the old model.

In [ ]:
# os.chdir("/content/drive/My Drive/Earth-Engine-with-Deep-Learning/models/pretrain/unet_v1")

In [ ]:
# %%writefile config.yaml
# autoScaling:
#     minNodes: 10


Writing config.yaml


In [ ]:
!gcloud ai-platform models create {Model_gs_name} --project {Project}

!gcloud ai-platform versions create {Version} \
    --project {Project} \
    --model {Model_gs_name} \
    --origin {EEified_path} \
    --framework "TENSORFLOW" \
    --runtime-version 2.2 \
    --python-version 3.7
    # --config=config.yaml